In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import sys
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
import time
from collections import Counter
tqdm.pandas(tqdm_notebook)
import lightgbm as lgb
import gc
import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression,Ridge,Lasso
import matplotlib.gridspec as gridspec
pd.set_option('display.float_format', '{:.5f}'.format)
SEED = 51
np.random.seed(SEED)
%matplotlib inline

In [2]:
pred1 = pd.read_csv('../my_submissions/ens.csv')
pred2 = pd.read_csv('../my_submissions/deneme5.csv')

In [3]:
negatives_1 = list(pred1[pred1['target'] < -15]['card_id'])

In [4]:
pred1[pred1['target'] < -15]

,card_id,target
7750,C_ID_a74b12dcf8,-18.71469
20556,C_ID_aae50409e7,-17.40968
21724,C_ID_92df9b9d44,-15.13480
25568,C_ID_767923bdb9,-16.69708
27982,C_ID_e7f772dfc0,-15.52800
29773,C_ID_aae5d02fad,-15.79106
32446,C_ID_ac114ef831,-19.44891
35851,C_ID_5ee52cf9f6,-17.36074
44491,C_ID_75519a6d74,-15.45048
47616,C_ID_eccc654bc9,-16.45724


In [5]:
print(len(negatives_1))

21


In [6]:
train = pd.read_csv('../input/train.csv')

In [7]:
#-33.21

In [8]:
pred2[pred2['target'] < -15]

,card_id,target
2726,C_ID_1edbae8171,-18.13505
5008,C_ID_be92f84f5c,-15.31044
6026,C_ID_91cc0c06ca,-15.72248
17778,C_ID_866eae8fb0,-17.61851
20556,C_ID_aae50409e7,-17.39345
23168,C_ID_3804897561,-15.45633
25568,C_ID_767923bdb9,-17.16466
26225,C_ID_44c140917d,-15.78383
27314,C_ID_7f5f950342,-15.76650
32446,C_ID_ac114ef831,-17.44147


In [9]:
negatives_2 = list(pred2[pred2['target'] < -15]['card_id'])

In [10]:
print(len(negatives_2))

30


In [11]:
[x for x in negatives_1 if x in negatives_2]

['C_ID_aae50409e7',
 'C_ID_767923bdb9',
 'C_ID_ac114ef831',
 'C_ID_5ee52cf9f6',
 'C_ID_cde027fde7',
 'C_ID_464246c0a5',
 'C_ID_6ab591cf62',
 'C_ID_922f9c5ea6',
 'C_ID_bced41d837',
 'C_ID_70c457436a',
 'C_ID_877b879853',
 'C_ID_86ddafb51c']

In [12]:
most_negatives = ['C_ID_aae50409e7',
 'C_ID_767923bdb9',
 'C_ID_ac114ef831',
 'C_ID_5ee52cf9f6',
 'C_ID_cde027fde7',
 'C_ID_464246c0a5',
 'C_ID_6ab591cf62',
 'C_ID_922f9c5ea6',
 'C_ID_bced41d837',
 'C_ID_70c457436a',
 'C_ID_877b879853',
 'C_ID_86ddafb51c']

In [13]:
def myfunc(x):
    a = x['card_id']
    b = x['target']
    if a in most_negatives:
        return -33.21
    else:
        return b

In [14]:
pred2['target'] = pred2.progress_apply(myfunc,axis=1)

100%|██████████| 123623/123623 [00:02<00:00, 46828.84it/s]


In [15]:
pred2['target'].min()

-33.21

In [17]:
pred2.head()

,card_id,target
0,C_ID_0ab67a22ab,-2.00503
1,C_ID_130fd0cbdd,-0.24504
2,C_ID_b709037bc5,-0.97021
3,C_ID_d27d835a9f,-0.14137
4,C_ID_2b5e3df5c2,-1.67992


In [18]:
pred2.to_csv('../my_submissions/deneme_son.csv',index=False)